In [54]:
import numpy as np
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
def search_string(formar_tag, latter_tag, text):
    """
    Find a keyword between specific tags or words.
    """
    lst = re.findall(r'{}.*?{}'.format(formar_tag, latter_tag), text, re.DOTALL)
    lst = [i.replace(formar_tag,'').replace(latter_tag,'')for i in lst]
    return lst

def remove_unnecessary_words(unnecessary_list,string):
    for word in unnecessary_list:
        string = re.sub(word,'',string)
    return string

def remove_tags_from_list(lst):
    lst = [remove_unnecessary_words(['<[^>]*>','\n'],str(i)) for i in lst]
    return lst

class Wantedly:
    def __init__(self,url):
        self.url= url
        self.requests = requests.get(self.url)
        self.html         = self.requests.text
        self.soup              = BeautifulSoup(self.html,'html.parser')

    def _get_project_source(self):
        self._project_source = self.soup.find_all('h1',{'class':'project-title'})

    def get_project_names(self):
        self._get_project_source()
        project_names = remove_tags_from_list(self._project_source)
        return project_names

    def get_project_urls(self,only_url=False):
        if only_url:
            self._get_project_source()
        domain = 'https://www.wantedly.com/'
        project_num_list = [search_string('<a href="/','filter', str(string))[0] for string in self._project_source]
        self.url_list = [domain + str(num) for num in project_num_list]
        return self.url_list

    def get_companies(self):
        companies =  self.soup.find_all('a',{'class':'wt-company'})
        companies = [link for link in companies if 'img' not in str(link)]
        companies = remove_tags_from_list(companies)
        return companies

In [55]:
url='https://www.wantedly.com/projects?type=recent&page=1&occupation_types%5B%5D=writer&hiring_types%5B%5D=contract'

In [56]:
print (url)

https://www.wantedly.com/projects?type=recent&page=1&occupation_types%5B%5D=writer&hiring_types%5B%5D=contract


In [57]:
wantedly = Wantedly(url)

In [58]:
wantedly.get_project_urls(only_url=True)

['https://www.wantedly.com/projects/247708?',
 'https://www.wantedly.com/projects/247288?',
 'https://www.wantedly.com/projects/247059?',
 'https://www.wantedly.com/projects/247123?',
 'https://www.wantedly.com/projects/219832?',
 'https://www.wantedly.com/projects/246912?',
 'https://www.wantedly.com/projects/246781?',
 'https://www.wantedly.com/projects/246743?',
 'https://www.wantedly.com/projects/245703?',
 'https://www.wantedly.com/projects/246080?']

In [59]:
wantedly.get_project_names()

['リモート可！急成長中！教育×テクノロジーメディア編集者WANTED！',
 '23歳社長直下で稼ぎまくれる！デジタル広告に革命を起こすライター募集',
 '趣味のネットサーフィンがライティングに生きる！？経歴不問のインターン募集！',
 'どこでも通用する執筆・取材力が身につく！学生マネーライターを募集！',
 '挑戦したい人募集！急成長中メディアの編集・戦略的ライターを WANTED！',
 '書くことが好き！マーケティングに興味のある好奇心旺盛なライター募集！',
 '世界中のクリエイターに支持されるメディアを一緒に立ち上げる編集長候補を募集',
 '反響営業で成り立つマーケティング力を持つ企業で編集兼ライター募集！',
 'あなたが大好きなセブ島を、思いのままに文章に起こすライターを募集！',
 '働くかっこいいを世界に伝える最強SEOライター募集']

In [60]:
wantedly.get_companies()

['キラメックス株式会社',
 '株式会社アキンドテック',
 '株式会社終活ねっと',
 '株式会社Money&amp;You',
 '株式会社ジーニング',
 '株式会社フィードフォース',
 'ThousandJapan株式会社',
 '株式会社リザーブリンク',
 'TNKトラベルJAPAN',
 'インビジョン株式会社']

In [61]:
import time

def apply_page_num(num,url):
    url = url.replace('page=1','page='+str(num+1))
    return url

def create_data_list(url,num_range):
    url_list = [apply_page_num(num, url) for num in range(num_range[0],num_range[1])]
    company_list          = []
    project_name_list     = []
    project_url_list      = [] 
    for url in url_list:
        instance = Wantedly(url)
        company_list = company_list + instance.get_companies()
        project_name_list    = project_name_list + instance.get_project_names()
        project_url_list     = project_url_list  + instance.get_project_urls()

    dictionary = {'company':company_list, 'project_name':project_name_list, 'project_url':project_url_list}
    df = pd.DataFrame.from_dict(dictionary)
    time.sleep(3)
    return df

In [62]:
df = create_data_list(url,(0,2))

In [78]:
df.head()

,company,project_name,project_url
0,キラメックス株式会社,リモート可！急成長中！教育×テクノロジーメディア編集者WANTED！,https://www.wantedly.com/projects/247708?
1,株式会社アキンドテック,23歳社長直下で稼ぎまくれる！デジタル広告に革命を起こすライター募集,https://www.wantedly.com/projects/247288?
2,株式会社終活ねっと,趣味のネットサーフィンがライティングに生きる！？経歴不問のインターン募集！,https://www.wantedly.com/projects/247059?
3,株式会社Money&amp;You,どこでも通用する執筆・取材力が身につく！学生マネーライターを募集！,https://www.wantedly.com/projects/247123?
4,株式会社ジーニング,挑戦したい人募集！急成長中メディアの編集・戦略的ライターを WANTED！,https://www.wantedly.com/projects/219832?


In [73]:
print(df)

              company                           project_name  \
0          キラメックス株式会社     リモート可！急成長中！教育×テクノロジーメディア編集者WANTED！   
1         株式会社アキンドテック     23歳社長直下で稼ぎまくれる！デジタル広告に革命を起こすライター募集   
2           株式会社終活ねっと  趣味のネットサーフィンがライティングに生きる！？経歴不問のインターン募集！   
3   株式会社Money&amp;You      どこでも通用する執筆・取材力が身につく！学生マネーライターを募集！   
4           株式会社ジーニング  挑戦したい人募集！急成長中メディアの編集・戦略的ライターを WANTED！   
5        株式会社フィードフォース     書くことが好き！マーケティングに興味のある好奇心旺盛なライター募集！   
6   ThousandJapan株式会社  世界中のクリエイターに支持されるメディアを一緒に立ち上げる編集長候補を募集   
7         株式会社リザーブリンク      反響営業で成り立つマーケティング力を持つ企業で編集兼ライター募集！   
8        TNKトラベルJAPAN      あなたが大好きなセブ島を、思いのままに文章に起こすライターを募集！   
9          インビジョン株式会社              働くかっこいいを世界に伝える最強SEOライター募集   
10         STUDIO株式会社       スタートアップSTUDIOを拡大させるインターンライター大募集！   
11          株式会社マイベスト  台湾華語ライター大募集！急成長メディアの台湾事業を一緒に盛り上げましょう！   
12           株式会社ukka    沖縄・離島から生産者のこだわりを全国に届ける産地ライター/FRを募集！   
13      株式会社TABI LABO       TABI LABOでWORLD TOPICSの外部ライター募集！   
14            株式会社DAT    未来に繋がるメディアを、新たな